In [158]:
import numpy as np
import pandas as pd
import cv2
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import importlib

import IrisLocalization
import IrisNormalization
import IrisEnhancement 
import IrisFeatureExtraction

importlib.reload(IrisLocalization)
importlib.reload(IrisNormalization)
importlib.reload(IrisEnhancement)
importlib.reload(IrisFeatureExtraction)

def generateFeatureVector(image):
    incir, outcir = Localization(image)
    img = Normalization(image, incir, outcir)
    img = Enhancement(img)
    V = FeatureExtraction(img)

    return V


def extractData(flag):
    '''
    flag: 1 if you want to extract train data, 2 if test data
    '''
    vecs = [] # list for stroring feature vectors
    source = 'CASIA Iris Image Database (version 1.0)/'

    n_eye = 108

    if flag == 1: # train data
        n = 3
    elif flag == 2: # test data
        n = 4

    for i in range(1, 2+1): # we need to change 2 -> n_eye!
        for j in range(1, n+1):
            path = source + '%03d' % (i,) + '/' + str(flag) + '/'
            filename = '%03d' % (i,) + '_' + str(flag) + '_' + str(j) + '.bmp'
            
            image = cv2.imread(path + filename)

            V = generateFeatureVector(image)

            vecs.append(V)
    
    vecs = np.array(vecs)

    return vecs



In [159]:
# Search closest iris among train dataset
# return two list, one of which stores closest iris and another stores their similarity measures
def IrisMatching(train, test):
    '''
    train: a list of feature vectors for train data
    test: a list of feature vectors for test data  
    '''
    train_X = train
    train_y = np.repeat(range(1,109),3)
    test_X = test
    test_y = np.repeat(range(1,109),4)

    # Implement LDA to improve omputational efficiency and classification accuracy 
    clf = LinearDiscriminantAnalysis()
    clf.fit(train_X, train_y)

    f = clf.transform(train_X)
    f_test = clf.transform(test_X)

    # Calculate Similarity Measures 
    for i in range(len(f_test)):
        
        df_min = pd.DataFrame(columns=['L1', 'L2', 'Cosine'])
        df_minmeasure = pd.DataFrame(columns=['L1', 'L2', 'Cosine'])

        for j in range(len(f)):
            d1 = sum(abs(f_test[i:i+1] - f[j:j+1]))
            d2 = sum((f_test[i:i+1] - f[j:j+1])**2)
            d3 = 1 - np.dot(f_test[i:i+1], f[j:j+1]) / np.sqrt(sum(f_test**2) * sum(f[j:j+1]**2))

            # Three Similarity Measures
            if mind1 > d1:
                mind1 = d1
                closest_d1 = j

            if mind2 > d2:
                mind2 = d2
                closest_d2 = j
            if mind3 > d3:
                mind3 = d3
                closest_d3 = j
            
        # Store outputs
        df_min.loc[i] = [closest_d1, closest_d2, closest_d3]
        df_minmeasure.loc[i] = [mind1, mind2, mind3]
    
    return df_min, df_minmeasure

        


In [160]:
train = extractData(1)
test = extractData(2)

IrisMatching(train, test)


IndexError: index 280 is out of bounds for axis 0 with size 280